In [79]:
import pandas as pd
import numpy as np
df=pd.read_csv('/content/mushrooms.csv')
df.head()


,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [80]:
X,y=df.drop(['class'],axis=1),df['class'].values
features_names=df.columns[1:]

In [73]:
features_names
# X.shape

Index(['cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
       'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color',
       'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
       'stalk-surface-below-ring', 'stalk-color-above-ring',
       'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number',
       'ring-type', 'spore-print-color', 'population', 'habitat'],
      dtype='object')

In [74]:
y.shape

(8124,)

In [75]:
y[X["cap-shape"]=="x"]

array(['p', 'e', 'p', ..., 'e', 'e', 'e'], dtype=object)

In [36]:
X["cap-shape"].unique()
(X["cap-shape"]=="x").sum()

3656

In [27]:
# from pandas.core.arrays.sparse import dtype
# def entropy(y):
#   classes=np.unique(y)
#   probs=np.empty((classes.shape[0]),dtype=np.float32)

#   for i ,cls in enumerate(classes):
#     probs[i]=(y==cls).mean()

#   return np.sum(-1*probs*np.log2(probs))


In [81]:
class DecisionTree():
  def __init__(self,max_depth=1,min_data_split=50):
    self.max_depth=max_depth
    self.min_data_split=min_data_split


  def fit(self,X,y):
    self.root=self.construct_node(X,y,0)

  def traverse_node(self,node,X):
    if node["is_leaf"]:
      return node["predections"]
    feat=node["column_selected"]
    feat_val=X[feat]
    return self.traverse_node(node["childrens"][feat_val],X)

  def predict(self,X):
    prediction=[]
    for i in range (len(X)):
      probs,classes=self.traverse_node(self.root,X.iloc[i])
      prediction.append(classes[np.argmax(probs)])
    return prediction



  def construct_node(self,X,y,curr_depth):
    if len(X)== 0 or len(X.columns)==0:
      return {}

    if curr_depth>self.max_depth:
      return {}

    node={"column_selected":None,
          "childrens":[],
          "is_leaf":False}
    if len(X)>= self.min_data_split:
      before_split_entropy=self.entropy(y)

      weighted_after_split_entropy=np.zeros((len(X.columns),))

      for i , feat in enumerate(X.columns):
        feat_vals=X[feat].unique()
        for feat_val in feat_vals:
          weighted_after_split_entropy[i]+=(X[feat]==feat_val).mean()*self.entropy(y[X[feat]==feat_val])

      node["column_selected"]=X.columns[np.argmax(before_split_entropy-weighted_after_split_entropy)]

      feat=node["column_selected"]
      feat_vals=X[feat].unique()

      for feat_val in feat_vals:
        X_split=X[X[feat]==feat_val].drop([feat],axis=1)
        y_split=y[X[feat]==feat_val]
        child_node=self.construct_node(X_split,y_split,curr_depth+1)
        if child_node=={}:
          node["is_leaf"]=True;
          node["childrens"]=[]
          break
        node["childrens"].append(child_node)
    else:
      node["is_leaf"]=True
    if node["is_leaf"]:
      node["predections"]=self.calc_probs(y)

    return node





  def calc_probs(self,y):
    classes=np.unique(y)
    probs=np.empty((classes.shape[0]),dtype=np.float32)
    for i ,cls in enumerate(classes):
      probs[i]=(y==cls).mean()
    return probs,classes


  def entropy(self , split_ix):
    probs,classes=self.calc_probs(y)
    return np.sum(-1*probs*np.log2(probs))


In [82]:
dt=DecisionTree()

In [83]:
x_test,y_test=X.iloc[-100:],y[-100]
x_train,y_train=X.iloc[:-100],y[:-100]

In [84]:
dt.fit(x_train,y_train)

In [87]:
dt.predict(x_test)

TypeError: ignored

In [66]:
from sklearn.preprocessing import LabelEncoder

In [68]:
le=LabelEncoder()
df.apply(le.fit)

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,0,3,2,4,0,5,0,0,0,11,...,2,5,5,0,1,1,4,0,1,2
8120,0,5,2,4,0,5,0,0,0,11,...,2,5,5,0,0,1,4,0,4,2
8121,0,2,2,4,0,5,0,0,0,5,...,2,5,5,0,1,1,4,0,1,2
8122,1,3,3,4,0,8,1,0,1,0,...,1,7,7,0,2,1,0,7,4,2
